[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aioaneia/semantic-papers-filter/blob/main/notebooks/exploration.ipynb)

# Semantic NLP Filter for Deep Learning Papers in Virology and Epidemiology

### **Clone** the repository https://github.com/aioaneia/semantic-papers-filter.git

In [ ]:
! git clone https://github.com/aioaneia/semantic-papers-filter.git

In [ ]:
! pip install kaleido

### Get the lastest code from main branch

In [ ]:

!cd semantic-papers-filter && git checkout main && git pull origin main


### Move to the source folder

In [ ]:
cd /content/semantic-papers-filter/src/

### Import modules

In [7]:
import sys
sys.path.append('/content/semantic-papers-filter/.')

from src.main import PaperFilteringApp

### Initialise the application.

In [ ]:
app = PaperFilteringApp('../config.yaml')

### Run the pipeline

In [ ]:
app.run()

### Show Statistics

In [ ]:
app.show_statistics()